In [3]:
import tensorflow as tf

#为名字为foo的命名空间创建名字为v的变量
with tf.variable_scope('foo'):
    v=tf.get_variable('v',[1],initializer=tf.constant_initializer(1.0))


#获取已经声明的变量
with tf.variable_scope('foo',reuse=True):
    v1=tf.get_variable('v',[1])
    print v==v1

True


In [4]:
#模型保存
import tensorflow as tf

v1_=tf.Variable(tf.constant(1.0,shape=[1]),name='v1_')
v2_=tf.Variable(tf.constant(2.0,shape=[1]),name='v2_')
result=v1_+v2_

saver=tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.save(sess,'./tmp/model_saver/model.ckpt')

In [1]:
#读取模型(jupyter kernel should restart!)
import tensorflow as tf

#使用和保存模型一样的方式来声明变量
v1_=tf.Variable(tf.constant(1.0,shape=[1]),name='v1_')
v2_=tf.Variable(tf.constant(2.0,shape=[1]),name='v2_')
result=v1_+v2_

saver=tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess,'./tmp/model_saver/model.ckpt')
    print sess.run(result)

INFO:tensorflow:Restoring parameters from ./tmp/model_saver/model.ckpt
[ 3.]


In [1]:
#读取模型(jupyter kernel should restart!)
import tensorflow as tf

saver=tf.train.import_meta_graph('./tmp/model_saver/model.ckpt.meta')

with tf.Session() as sess:
    saver.restore(sess,'./tmp/model_saver/model.ckpt')
    
    #通过张量名称来获取张量
    print sess.run(tf.get_default_graph().get_tensor_by_name('v1_:0'))

INFO:tensorflow:Restoring parameters from ./tmp/model_saver/model.ckpt
[ 1.]


In [2]:
#page113
#读取模型(jupyter kernel should restart!)
"""
比如，可能有一个之前训练好的五层神经网络模型，
当现在想尝试一个六层的神经网络，
那么可以将前面五层神经网络中的参数直接加载到新的模型，
而仅仅将最后一层神经网络重新训练
"""

import tensorflow as tf


# saver=tf.train.import_meta_graph('./tmp/model_saver/model.ckpt.meta')
#加载部分变量
v1_=tf.Variable(tf.constant(1.0,shape=[1]),name='v2_')
saver=tf.train.Saver([v1_])

#加载变量并重命名
# v1_=tf.Variable(tf.constant(1.0,shape=[1]),name='v2_rename')
# saver=tf.train.Saver(['v1_':v1_])


with tf.Session() as sess:
    saver.restore(sess,'./tmp/model_saver/model.ckpt')
    
    #通过张量名称来获取张量
    print sess.run(tf.get_default_graph().get_tensor_by_name('v2_:0'))


INFO:tensorflow:Restoring parameters from ./tmp/model_saver/model.ckpt
[ 2.]


### 加载变量并重命名的使用场景：
在滑动平均模型中，加载模型时直接将影子变量映射到变量自身，那么在使用训练好的模型时就不需要再调用函数来获取变量的滑动平均值，样例如下：

In [1]:
import tensorflow as tf

v=tf.Variable(0,dtype=tf.float32,name='v')

for variables in tf.global_variables():
    print variables.name
    """
    v:0
    """
    
ema=tf.train.ExponentialMovingAverage(0.99)
maintain_averages_op=ema.apply(tf.global_variables())
#声明滑动平均模型之后，Tensorflow会自动生成一个影子变量
for variables in tf.global_variables():
    print variables.name
    """
    v:0
    v:0
    v/ExponentialMovingAverage:0
    """
    
saver=tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    sess.run(tf.assign(v,10))
    sess.run(maintain_averages_op)
    saver.save(sess,'./tmp/variable_rename/model.ckpt')
    """
    v:0   v/ExponentialMovingAverage:0
    这两个变量都会保存下来
    """
    print sess.run([v,ema.average(v)])
    

v:0
v:0
v/ExponentialMovingAverage:0
[10.0, 0.099999905]


In [2]:
import tensorflow as tf

v=tf.Variable(0,dtype=tf.float32,name='v')

#通过变量重命名将原来的v的滑动平均值赋值给v
saver=tf.train.Saver({'v/ExponentialMovingAverage':v})
with tf.Session() as sess:
    saver.restore(sess,'./tmp/variable_rename/model.ckpt')
    print sess.run(v)#输出值便是滑动平均值
    

INFO:tensorflow:Restoring parameters from ./tmp/variable_rename/model.ckpt
0.0999999


In [2]:
#加载滑动平均值的一种便捷方法
import tensorflow as tf

v=tf.Variable(0,dtype=tf.float32,name='v')
ema=tf.train.ExponentialMovingAverage(0.99)

#此函数可以直接生成重命名所需要的字典
print ema.variables_to_restore()

saver=tf.train.Saver(ema.variables_to_restore())

with tf.Session() as sess:
    saver.restore(sess,'./tmp/variable_rename/model.ckpt')
    print sess.run(v)

{u'v/ExponentialMovingAverage': <tf.Variable 'v:0' shape=() dtype=float32_ref>}
INFO:tensorflow:Restoring parameters from ./tmp/variable_rename/model.ckpt
0.0999999


 ### convert_variables_to_constants运用场景
 tf.train.Saver()会保存全部信息；而在测试、离线预测或者迁移学习的时候，只需要知道如何从神经网络的输入层经过向前传播计算得到输出成即可。
 利用convert_variables_to_constants方法可以将Tensorflow计算图统一存放在一个文件中，这样也更方便。样例如下：

In [2]:
#存储
import tensorflow as tf
from tensorflow.python.framework import graph_util

v1=tf.Variable(tf.constant(1.0,shape=[1]),name='v1')
v2=tf.Variable(tf.constant(2.0,shape=[1]),name='v2')

result=v1+v2

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    graph_def=tf.get_default_graph().as_graph_def()
    
    out_graph_def=graph_util.convert_variables_to_constants(
        sess,
        graph_def,
        ['add']
    )
    
    with tf.gfile.GFile('./tmp/convert_vtoc/model.pd','wb') as f:
        f.write(out_graph_def.SerializeToString())
    

INFO:tensorflow:Froze 2 variables.
Converted 2 variables to const ops.


In [1]:
#读取
import tensorflow as tf
from tensorflow.python.platform import gfile

with tf.Session() as sess:
    model_filename='./tmp/convert_vtoc/model.pd'
    with gfile.FastGFile(model_filename,'rb') as f:
        graph_def=tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    result=tf.import_graph_def(graph_def,return_elements=['add:0'])
    print sess.run(result)

[array([ 3.], dtype=float32)]


In [3]:
#持久化原理与数据格式
import tensorflow as tf

v1=tf.Variable(tf.constant(1.0,shape=[1]),name='v1')
v2=tf.Variable(tf.constant(1.0,shape=[1]),name='v2')
result=v1+v2

saver=tf.train.Saver()
saver.export_meta_graph('./tmp/meta_data/model.ckpt.meta.json',as_text=True)

meta_info_def {
  stripped_op_list {
    op {
      name: "Add"
      input_arg {
        name: "x"
        type_attr: "T"
      }
      input_arg {
        name: "y"
        type_attr: "T"
      }
      output_arg {
        name: "z"
        type_attr: "T"
      }
      attr {
        name: "T"
        type: "type"
        allowed_values {
          list {
            type: DT_HALF
            type: DT_FLOAT
            type: DT_DOUBLE
            type: DT_UINT8
            type: DT_INT8
            type: DT_INT16
            type: DT_INT32
            type: DT_INT64
            type: DT_COMPLEX64
            type: DT_COMPLEX128
            type: DT_STRING
          }
        }
      }
    }
    op {
      name: "Assign"
      input_arg {
        name: "ref"
        type_attr: "T"
        is_ref: true
      }
      input_arg {
        name: "value"
        type_attr: "T"
      }
      output_arg {
        name: "output_ref"
        type_attr: "T"
        is_ref: true
      }
      att